In [1]:
#importar las librerias
import sys
import os
import re
sys.path.append(os.path.abspath('../scripts'))  #esto lo colocamos para que el python nos lea la ruta bien
from utils import process_null_values  # Importa la funcinn desde utils
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords

In [2]:
CellReviews = pd.read_json('../data/Cell_Phones_and_Accessories_5.json', lines=True)

In [3]:
#acercamiento a los datos numericos
CellReviews.describe()

,overall,unixReviewTime
count,194439.000000,1.944390e+05
mean,4.129912,1.368714e+09
std,1.222499,3.230032e+07
min,1.000000,9.828000e+08
25%,4.000000,1.357603e+09
50%,5.000000,1.374538e+09
75%,5.000000,1.390262e+09
max,5.000000,1.406074e+09


In [4]:
nltk.download('punkt')  # Tokenizador de NLTK
nltk.download('wordnet')  # Diccionario de WordNet

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
CellReviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [7]:
#tipos de datos de los conjuntos de nuestro dataframe
CellReviews.dtypes

reviewerID        object
asin              object
reviewerName      object
helpful           object
reviewText        object
overall            int64
summary           object
unixReviewTime     int64
reviewTime        object
dtype: object

## ZONA DE TRABAJO FORZOZOS


In [15]:
#ANTES AQUI HABIA ALGO

In [5]:
process_null_values(CellReviews)

Total de registros:
la columna reviewerName tiene un Total de: 3519 valores vacios
--------------------------------------------------------------------------------
Porcentaje de registros:
la columna reviewerName tiene un Porcentaje de: 1.8098221035903292 valores vacios
--------------------------------------------------------------------------------
Si las columnas son irrelevantes, podemos hacer algo con sus datos, que te gustaria eliminarlos o los conservamos?
columna reviewerName eliminada exitosamente
data limpiada maestro, que tengas un buen dia :)


In [6]:
CellReviews.describe()

,overall,unixReviewTime
count,194439.000000,1.944390e+05
mean,4.129912,1.368714e+09
std,1.222499,3.230032e+07
min,1.000000,9.828000e+08
25%,4.000000,1.357603e+09
50%,5.000000,1.374538e+09
75%,5.000000,1.390262e+09
max,5.000000,1.406074e+09


In [7]:
#Valores duplicados????
CellReviews[CellReviews['unixReviewTime'].duplicated(keep=False)]

,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
...,...,...,...,...,...,...,...,...
194434,A1YMNTFLNDYQ1F,B00LORXVUE,"[0, 0]",Works great just like my original one. I reall...,5,This works just perfect!,1405900800,"07 21, 2014"
194435,A15TX8B2L8B20S,B00LORXVUE,"[0, 0]",Great product. Great packaging. High quality a...,5,Great replacement cable. Apple certified,1405900800,"07 21, 2014"
194436,A3JI7QRZO1QG8X,B00LORXVUE,"[0, 0]","This is a great cable, just as good as the mor...",5,Real quality,1405900800,"07 21, 2014"
194437,A1NHB2VC68YQNM,B00LORXVUE,"[0, 0]",I really like it becasue it works well with my...,5,I really like it becasue it works well with my...,1405814400,"07 20, 2014"


In [8]:
CellReviews.drop(['asin'], axis=1, inplace=True) #eliminando una columna inesesaria
CellReviews.drop(['summary'], axis=1, inplace=True) #eliminando una columna inesesaria
CellReviews.drop(['reviewTime'], axis=1, inplace=True) #eliminando una columna inesesaria
CellReviews.drop(['helpful'], axis=1, inplace=True) #eliminando una columna inesesaria
CellReviews.drop(['reviewerID'], axis=1, inplace=True) #eliminando una columna inesesaria
CellReviews.drop(['unixReviewTime'], axis=1, inplace=True) #eliminando una columna inesesaria


In [15]:
#antes de pasar a hacer la lematizacion hare una eliminacion de muchos datos positivos que hay
#eliminar muchos registros para balancear la data
condition = CellReviews['overall'] == 5

# Número de registros que deseas eliminar
num_to_remove = 8000

# Filtrar los primeros 'num_to_remove' registros que cumplan con la condición
rows_to_remove = CellReviews[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
CellReviews = CellReviews.drop(rows_to_remove.index)

CellReviews.describe()



,overall
count,137439.000000
mean,3.769061
std,1.292337
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [18]:
#funcion de lematizacion y tokenizacion
# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

# Obtener stopwords en inglés
stop_words = set(stopwords.words('english'))

#customizarlos
customs_stop_wards = stop_words - {"not", "no", "never", "ever"}#dejamos habilitadas estas palabras

def tokenice_and_lemati(text):
    tokens = word_tokenize(text.lower())#convertimos todo a minusc y luego lo tokenizamos jijijij
    filter_tokens = [token for token in tokens if token not in customs_stop_wards]
    lematized_tokens = [lemmatizer.lemmatize(token, pos='v') for token in filter_tokens]
    return ' '.join(lematized_tokens)
    

In [17]:
#quitar carecteres especiales
CellReviews['reviewText'] = CellReviews['reviewText'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

In [19]:
# Seleccionar un subconjunto aleatorio de 5 filas
small_data = CellReviews.sample(n=10000, random_state=42)
small_data.describe()


,overall
count,10000.000000
mean,3.830700
std,1.338064
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [19]:
CellReviews['processed_text'] = CellReviews['reviewText'].apply(tokenice_and_lemati)

In [20]:
CellReviews.head(10)

,reviewText,overall,processed_text
0,They look good and stick good I just dont like...,4,look good stick good dont like round shape alw...
3,Item arrived in great time and was in perfect ...,4,item arrive great time perfect condition howev...
5,These make using the home button easy My daugh...,3,make use home button easy daughter like would ...
7,it worked for the first week then it only char...,1,work first week charge phone 20 waste money
19,It worked great for the first couple of weeks ...,1,work great first couple weeks stop completely ...
22,does not have the need amps to charge things l...,3,not need amps charge things like ipads hp touc...
23,They are nothing special for sure but its nice...,4,nothing special sure nice tell power lead glow...
24,I have several chargers Have more than one veh...,4,several chargers one vehicle keep one nothing ...
26,I am disappointed that the 1A didnt work with ...,2,disappoint 1a didnt work ipad thats get buy ch...
27,This is a nice charger but you can tell it was...,4,nice charger tell make cheaply china charge ph...


In [ ]:
#visualizacion de la data
CellReviews.drop(['reviewText'], axis=1, inplace=True) #eliminando una columna inesesaria



,overall,processed_text
0,4,look good stick good dont like round shape alw...
3,4,item arrive great time perfect condition howev...
5,3,make use home button easy daughter like would ...
7,1,work first week charge phone 20 waste money
19,1,work great first couple weeks stop completely ...


In [25]:
#eliminando duplicados
CellReviews[CellReviews['processed_text'].duplicated(keep=False)]
# Eliminar duplicados basados en la columna 'processed_text'
CellReviews = CellReviews.drop_duplicates(subset=['processed_text'])


In [ ]:
CellReviews[CellReviews['processed_text'].duplicated(keep=False)]

,overall,processed_text


In [27]:
#guardar la data
CellReviews.to_csv('../data_process/cellphones.csv', index=False)
